# Make a movie of the NREL5MW turbine

In [1]:
# Add any possible locations of amr-wind-frontend here
amrwindfedirs = ['/projects/wind_uq/lcheung/amrwind-frontend/',
                  '/ccs/proj/cfd162/lcheung/amrwind-frontend/',
                ]
import sys, os, shutil, io
import numpy as np
for x in amrwindfedirs: sys.path.insert(1, x)

from functools import partial
import matplotlib.pyplot as plt
import pandas as pd

# Make all plots inline 
#%matplotlib inline

import postproengine as ppeng

import imageio
import glob
import cv2

/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
# Load ruamel or pyyaml as needed
try:
    import ruamel.yaml
    yaml = ruamel.yaml.YAML(typ='unsafe', pure=True)
    useruamel=True
    loaderkwargs = {'Loader':yaml.RoundTripLoader}
    dumperkwargs = {'Dumper':yaml.RoundTripDumper, 'indent':4, 'default_flow_style':False}
    Loader=yaml.load
except:
    import yaml as yaml
    useruamel=False
    loaderkwargs = {}
    dumperkwargs = {'default_flow_style':False }
    Loader=yaml.safe_load
    
def stringReplaceDict(s, dreplace):
    outstr = str(s)
    for k, g in dreplace.items():
        outstr=outstr.replace(k, str(g))
    return outstr
    
def formatTfig(fig,ax):
    ax.set_xlim([300, 301])
    return

def addturbines(fig, ax):
    outputcsv = '/pscratch/lcheung/AWAKEN/Benchmark1/Phase2/FarmRuns/prod1/BM2_FarmRunProd1newBC/turbines_farmrun.csv'
    # Process the csv file
    df = pd.read_csv(outputcsv)
    # Fix the turbine types of the turbines
    for index, row in df.iterrows():
        ax.plot(row[' x'], row[' y'], 's', color='k', markersize=1)
    return

#ppeng.formatTfig = formatTfig
#ppeng.addturbines = addturbines

In [3]:
replacedict={'RUNDIR':'/gpfs/lcheung/HFM/exawind-benchmarks/NREL5MW_ALM_BD_OFv402_ROSCO/',
            }

In [4]:
yamlstring="""
globalattributes:
  verbose: False
  executeorder:
  - instantaneousplanes

instantaneousplanes:
  - name: HHplot
    ncfile: RUNDIR/post_processing/turbhh_30000.nc
    iters:  [-1]
    times: [15300, 15301, 15302, 15303, 15304, 15305]
    iplane: 0
    xaxis: a1
    yaxis: a2
    # --- optional stuff here ---
    plot:
      title: 'ABL $U_h$ T={time:0.1f}s'
      plotfunc: "lambda db,i: np.sqrt(db['velocityx'][i]**2 + db['velocityy'][i]**2)"
      clevels: 'np.linspace(0, 14, 141)'
      #plotfunc: "lambda db,i: db['velocityz'][i]"
      #clevels: 'np.linspace(-5, 5, 41)'
      #postplotfunc: postproengine.addturbines
      xscalefunc: 'lambda x: (x-126*4)/126.0'
      yscalefunc: 'lambda y: (y-126*2)/126.0'
      xlabel: 'X/D'
      ylabel: 'Y/D'
      cbar_nticks: 8
      fontsize: 10
      figsize: [8,3]
      dpi: 125
      plotturbines:
      - basexyz: [0,0,0]
        hubheight: 1.1
        rotordiameter: 1
        nacelledir: 270
      savefile: movieframes/NREL5MW_wake_z90_{time:0.1f}.png
    animate:
      name: ../results/images/NREL5MW_movie.mp4
      fps: 5
      imagefilename: movieframes/NREL5MW_wake_z90_{time:0.1f}.png
"""
f = io.StringIO(stringReplaceDict(yamlstring, replacedict))
yamldict = Loader(f, **loaderkwargs)
yamldict['instantaneousplanes'][0]['times']=np.arange(15300, 15360, 1)

In [5]:
%%capture
# Run the driver
ppeng.driver(yamldict, verbose=True)

In [6]:
# Create an animated GIF from the movie frames
allframes=glob.glob('movieframes/*.png')
images=[]
for f in allframes:
    img = cv2.imread(f)
    if img is not None:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img_rgb)
imageio.mimsave('../results/images/NREL5MW_movie.gif', images, fps=5)